# Filtering: The Building Blocks of Motion Energy Models

## Overview

Filtering is a fundamental operation in signal processing and a core component of motion energy models. In this section, we'll explore different types of filters and their applications in motion detection.

### What we'll cover:
- Low-pass, high-pass, and band-pass filters
- Spatial and temporal filtering
- Gabor filters and their properties
- Implementing and applying filters to visual stimuli
- The relationship between filtering and neural processing

## Setting Up

Let's import the libraries we'll need for this section.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import scipy.signal as signal
from scipy.fft import fft, fft2, fftshift
import sys

# Add the utils package to the path
sys.path.append('../../..')
try:
    from motionenergy.utils import stimuli_generation, visualization, filtering
except ImportError:
    print("Note: utils modules not found. This is expected if you haven't implemented them yet.")

# For interactive plots
%matplotlib inline
from IPython.display import HTML, display

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. Introduction to Filtering

Filtering is the process of selectively enhancing or suppressing certain aspects of a signal. In visual processing, filters can be used to extract specific features from images or videos, such as edges, textures, or motion.

### What is a Filter?

A filter is characterized by its impulse response (how it responds to a brief input pulse) or, equivalently, by its frequency response (how it affects different frequencies). Mathematically, filtering is implemented through convolution of the input signal with the filter's impulse response.

### Types of Filters

Filters can be classified based on which frequencies they allow to pass through:

- **Low-pass filters**: Allow low frequencies and attenuate high frequencies
- **High-pass filters**: Allow high frequencies and attenuate low frequencies
- **Band-pass filters**: Allow a specific range of frequencies and attenuate others
- **Band-stop filters**: Attenuate a specific range of frequencies and allow others

Let's visualize some common filters and their effects on visual stimuli:

In [ ]:
def plot_filter_response(filter_type, cutoff_freq=0.2, order=5):
    """
    Plot the impulse response and frequency response of a 1D filter.
    
    Parameters:
    -----------
    filter_type : str
        Type of filter ('lowpass', 'highpass', 'bandpass', 'bandstop')
    cutoff_freq : float or tuple
        Cutoff frequency (normalized), or tuple of (low, high) for bandpass/bandstop
    order : int
        Filter order
    """
    # Create the filter using scipy.signal
    if filter_type == 'lowpass':
        b, a = signal.butter(order, cutoff_freq, btype='low')
        title = f"Low-pass Filter (cutoff={cutoff_freq}, order={order})"
    elif filter_type == 'highpass':
        b, a = signal.butter(order, cutoff_freq, btype='high')
        title = f"High-pass Filter (cutoff={cutoff_freq}, order={order})"
    elif filter_type == 'bandpass':
        if not isinstance(cutoff_freq, tuple):
            cutoff_freq = (cutoff_freq, 2*cutoff_freq)
        b, a = signal.butter(order, cutoff_freq, btype='band')
        title = f"Band-pass Filter (cutoff={cutoff_freq}, order={order})"
    elif filter_type == 'bandstop':
        if not isinstance(cutoff_freq, tuple):
            cutoff_freq = (cutoff_freq, 2*cutoff_freq)
        b, a = signal.butter(order, cutoff_freq, btype='stop')
        title = f"Band-stop Filter (cutoff={cutoff_freq}, order={order})"
    else:
        raise ValueError("Invalid filter type")
    
    # Compute the impulse response (time domain)
    impulse = np.zeros(100)
    impulse[0] = 1.0
    response = signal.lfilter(b, a, impulse)
    
    # Compute the frequency response
    w, h = signal.freqz(b, a)
    
    # Create example signals
    t = np.linspace(0, 1, 1000, endpoint=False)
    # High-frequency signal
    high_freq = np.sin(2 * np.pi * 10 * t)
    # Low-frequency signal
    low_freq = np.sin(2 * np.pi * 2 * t)
    # Mixed signal
    mixed = high_freq + low_freq
    
    # Apply the filter to the mixed signal
    filtered = signal.lfilter(b, a, mixed)
    
    # Plot the results
    fig, axes = plt.subplots(3, 1, figsize=(12, 10))
    
    # Impulse response
    axes[0].stem(np.arange(len(response)), response, use_line_collection=True)
    axes[0].set_title(f"Impulse Response of {title}")
    axes[0].set_xlabel('Sample')
    axes[0].set_ylabel('Amplitude')
    
    # Frequency response
    axes[1].plot(w / np.pi, np.abs(h))
    axes[1].set_title(f"Frequency Response of {title}")
    axes[1].set_xlabel('Normalized Frequency (×π rad/sample)')
    axes[1].set_ylabel('Magnitude')
    axes[1].grid(True)
    
    # Example signal filtering
    axes[2].plot(t[:200], mixed[:200], 'b-', alpha=0.5, label='Mixed Signal')
    axes[2].plot(t[:200], filtered[:200], 'r-', label='Filtered Signal')
    axes[2].set_title(f"Effect of {title} on a Mixed Signal")
    axes[2].set_xlabel('Time (s)')
    axes[2].set_ylabel('Amplitude')
    axes[2].legend()
    
    plt.tight_layout()
    return fig

# Visualize different types of filters
plot_filter_response('lowpass', cutoff_freq=0.2)

In [ ]:
# High-pass filter
plot_filter_response('highpass', cutoff_freq=0.1)

In [ ]:
# Band-pass filter
plot_filter_response('bandpass', cutoff_freq=(0.1, 0.4))

In [ ]:
# Band-stop filter
plot_filter_response('bandstop', cutoff_freq=(0.1, 0.4))

## 2. Spatial Filtering in Images

Spatial filtering is the application of filters to image data to enhance or suppress specific spatial features. Common applications include edge detection, noise reduction, and texture analysis.

Let's create some example spatial filters and see their effects on images:

In [ ]:
def create_test_image(size=128):
    """
    Create a test image with various spatial features.
    """
    # Create a blank image
    image = np.zeros((size, size))
    
    # Add a square
    image[size//4:3*size//4, size//4:3*size//4] = 0.5
    
    # Add a smaller square with higher intensity
    image[size//3:2*size//3, size//3:2*size//3] = 1.0
    
    # Add some diagonal lines
    for i in range(size):
        if i % 20 < 5:
            j = min(i, size-1)
            image[j, i] = 1.0
            image[i, j] = 1.0
    
    # Add a circular region
    center = size // 2
    radius = size // 8
    y, x = np.ogrid[-center:size-center, -center:size-center]
    mask = x*x + y*y <= radius*radius
    image[mask] = 0.8
    
    return image

# Create a test image
test_image = create_test_image(128)

# Display the test image
plt.figure(figsize=(8, 8))
plt.imshow(test_image, cmap='gray')
plt.title('Test Image')
plt.colorbar()
plt.axis('off')
plt.show()

In [ ]:
def apply_spatial_filter(image, filter_name):
    """
    Apply a spatial filter to an image.
    
    Parameters:
    -----------
    image : ndarray
        Input image
    filter_name : str
        Name of the filter to apply
    
    Returns:
    --------
    filtered_image : ndarray
        Filtered image
    kernel : ndarray
        Filter kernel used
    """
    if filter_name == 'box_blur':
        # Box blur filter (low-pass)
        kernel = np.ones((5, 5)) / 25
        title = "Box Blur Filter (Low-pass)"
    elif filter_name == 'gaussian_blur':
        # Gaussian blur filter (low-pass)
        sigma = 2.0
        size = int(2 * np.ceil(3 * sigma) + 1)
        x = np.arange(-(size//2), size//2 + 1)
        y = np.arange(-(size//2), size//2 + 1)
        X, Y = np.meshgrid(x, y)
        kernel = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
        kernel = kernel / np.sum(kernel)  # Normalize
        title = f"Gaussian Blur Filter (Low-pass, sigma={sigma})"
    elif filter_name == 'edge_detection':
        # Laplacian edge detection filter (high-pass)
        kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
        title = "Laplacian Edge Detection Filter (High-pass)"
    elif filter_name == 'sobel_x':
        # Sobel X gradient filter (edge detection)
        kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        title = "Sobel X Gradient Filter"
    elif filter_name == 'sobel_y':
        # Sobel Y gradient filter (edge detection)
        kernel = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
        title = "Sobel Y Gradient Filter"
    elif filter_name == 'sharpen':
        # Sharpening filter
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        title = "Sharpening Filter"
    else:
        raise ValueError(f"Unknown filter name: {filter_name}")
    
    # Apply the filter using convolution
    filtered_image = signal.convolve2d(image, kernel, mode='same', boundary='symm')
    
    # For visualization purposes, clip and normalize the filtered image
    if filter_name in ['edge_detection', 'sobel_x', 'sobel_y']:
        # For edge detection filters, take the absolute value and normalize
        filtered_image = np.abs(filtered_image)
        if np.max(filtered_image) > 0:
            filtered_image = filtered_image / np.max(filtered_image)
    
    return filtered_image, kernel, title

# Apply different spatial filters to the test image
filter_names = ['box_blur', 'gaussian_blur', 'edge_detection', 'sobel_x', 'sobel_y', 'sharpen']
filtered_images = {}

for filter_name in filter_names:
    filtered_image, kernel, title = apply_spatial_filter(test_image, filter_name)
    filtered_images[filter_name] = (filtered_image, kernel, title)

# Visualize the filtered images
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

# Original image
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Filtered images
for i, filter_name in enumerate(filter_names, 1):
    filtered_image, kernel, title = filtered_images[filter_name]
    
    # Show the filtered image
    axes[i].imshow(filtered_image, cmap='gray')
    axes[i].set_title(title)
    axes[i].axis('off')
    
    # Show the kernel as an inset
    ax_inset = axes[i].inset_axes([0.05, 0.05, 0.3, 0.3])
    ax_inset.imshow(kernel, cmap='RdBu')
    ax_inset.set_title('Kernel')
    ax_inset.set_xticks([])
    ax_inset.set_yticks([])

# Hide the last subplot if the number of filters is not 8
if len(filter_names) < 8:
    axes[8].axis('off')

plt.tight_layout()
plt.show()

## 3. Temporal Filtering

Temporal filtering operates on time-varying signals to enhance or suppress specific temporal features. In the context of motion processing, temporal filters can be used to detect changes in a scene over time.

Let's look at some basic temporal filters and their effects on time-varying signals:

In [ ]:
def create_temporal_signal(duration=2.0, sampling_rate=1000):
    """
    Create a temporal signal with different frequency components.
    
    Parameters:
    -----------
    duration : float
        Duration of the signal in seconds
    sampling_rate : int
        Sampling rate in Hz
    
    Returns:
    --------
    t : ndarray
        Time points
    signal : ndarray
        The temporal signal
    """
    t = np.linspace(0, duration, int(duration * sampling_rate), endpoint=False)
    
    # Create a signal with multiple frequency components
    # Low frequency component (1 Hz)
    low_freq = np.sin(2 * np.pi * 1 * t)
    
    # Medium frequency component (5 Hz)
    medium_freq = 0.5 * np.sin(2 * np.pi * 5 * t)
    
    # High frequency component (20 Hz)
    high_freq = 0.25 * np.sin(2 * np.pi * 20 * t)
    
    # Transient component (short pulse)
    transient = 0.5 * (np.exp(-((t - duration/2)**2) / (2 * 0.01**2)))
    
    # Combine the components
    signal = low_freq + medium_freq + high_freq + transient
    
    return t, signal

# Create a temporal signal
t, signal = create_temporal_signal()

# Plot the signal
plt.figure(figsize=(12, 4))
plt.plot(t, signal)
plt.title('Example Temporal Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

In [ ]:
def apply_temporal_filter(signal, sampling_rate, filter_name):
    """
    Apply a temporal filter to a signal.
    
    Parameters:
    -----------
    signal : ndarray
        Input signal
    sampling_rate : int
        Sampling rate in Hz
    filter_name : str
        Name of the filter to apply
    
    Returns:
    --------
    filtered_signal : ndarray
        Filtered signal
    title : str
        Filter description
    """
    nyquist = 0.5 * sampling_rate
    
    if filter_name == 'low_pass':
        # Low-pass filter: keeps slow changes, removes rapid oscillations
        cutoff = 3.0  # 3 Hz cutoff
        b, a = signal.butter(5, cutoff / nyquist, btype='low')
        title = f"Low-pass Filter (cutoff={cutoff} Hz)"
    elif filter_name == 'high_pass':
        # High-pass filter: removes slow changes, keeps rapid oscillations
        cutoff = 10.0  # 10 Hz cutoff
        b, a = signal.butter(5, cutoff / nyquist, btype='high')
        title = f"High-pass Filter (cutoff={cutoff} Hz)"
    elif filter_name == 'band_pass':
        # Band-pass filter: keeps oscillations within a frequency range
        low_cutoff = 4.0  # 4 Hz lower cutoff
        high_cutoff = 8.0  # 8 Hz upper cutoff
        b, a = signal.butter(5, [low_cutoff / nyquist, high_cutoff / nyquist], btype='band')
        title = f"Band-pass Filter (cutoff={low_cutoff}-{high_cutoff} Hz)"
    elif filter_name == 'derivative':
        # Temporal derivative: enhances rapid changes (edges in time)
        # Simple first-order difference
        filtered_signal = np.zeros_like(signal)
        filtered_signal[1:] = signal[1:] - signal[:-1]
        return filtered_signal, "Temporal Derivative Filter"
    else:
        raise ValueError(f"Unknown filter name: {filter_name}")
    
    # Apply the filter
    filtered_signal = signal.filtfilt(b, a, signal)
    
    return filtered_signal, title

# Apply different temporal filters to the signal
filter_names = ['low_pass', 'high_pass', 'band_pass', 'derivative']
filtered_signals = {}

for filter_name in filter_names:
    filtered_signal, title = apply_temporal_filter(signal, 1000, filter_name)
    filtered_signals[filter_name] = (filtered_signal, title)

# Visualize the filtered signals
fig, axes = plt.subplots(len(filter_names) + 1, 1, figsize=(12, 12))

# Original signal
axes[0].plot(t, signal)
axes[0].set_title('Original Signal')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True)

# Filtered signals
for i, filter_name in enumerate(filter_names, 1):
    filtered_signal, title = filtered_signals[filter_name]
    axes[i].plot(t, filtered_signal)
    axes[i].set_title(title)
    axes[i].set_xlabel('Time (s)')
    axes[i].set_ylabel('Amplitude')
    axes[i].grid(True)

plt.tight_layout()
plt.show()

## 4. Gabor Filters

Gabor filters are a special class of filters that play a crucial role in motion energy models. They are sinusoidal gratings modulated by a Gaussian envelope, which makes them particularly effective at detecting oriented patterns at specific spatial frequencies.

The 2D Gabor function is defined as:

$$g(x, y; \lambda, \theta, \psi, \sigma, \gamma) = \exp\left(-\frac{x'^2 + \gamma^2 y'^2}{2\sigma^2}\right) \cos\left(2\pi\frac{x'}{\lambda} + \psi\right)$$

where:
- $x' = x\cos\theta + y\sin\theta$
- $y' = -x\sin\theta + y\cos\theta$
- $\lambda$ is the wavelength (inverse of spatial frequency)
- $\theta$ is the orientation
- $\psi$ is the phase offset
- $\sigma$ is the standard deviation of the Gaussian envelope
- $\gamma$ is the spatial aspect ratio

Let's implement and visualize Gabor filters:

In [ ]:
def gabor_filter(size, lambda_val, theta, psi, sigma, gamma):
    """
    Create a 2D Gabor filter.
    
    Parameters:
    -----------
    size : int
        Size of the filter (size x size)
    lambda_val : float
        Wavelength of the sinusoidal component
    theta : float
        Orientation of the Gabor filter in radians
    psi : float
        Phase offset in radians
    sigma : float
        Standard deviation of the Gaussian envelope
    gamma : float
        Spatial aspect ratio
    
    Returns:
    --------
    gabor : ndarray
        2D Gabor filter
    """
    # Create the grid
    x = np.linspace(-size//2, size//2, size)
    y = np.linspace(-size//2, size//2, size)
    X, Y = np.meshgrid(x, y)
    
    # Rotation
    X_theta = X * np.cos(theta) + Y * np.sin(theta)
    Y_theta = -X * np.sin(theta) + Y * np.cos(theta)
    
    # Compute the Gabor filter
    gb = np.exp(-(X_theta**2 + gamma**2 * Y_theta**2) / (2 * sigma**2)) * \
         np.cos(2 * np.pi * X_theta / lambda_val + psi)
    
    return gb

# Create Gabor filters with different parameters
def plot_gabor_filters():
    """
    Create and visualize Gabor filters with different parameters.
    """
    # Base parameters
    size = 64
    lambda_val = 10.0
    theta = 0.0
    psi = 0.0
    sigma = 10.0
    gamma = 1.0
    
    # Generate filters with different orientations
    orientations = [0, np.pi/6, np.pi/4, np.pi/3, np.pi/2, 2*np.pi/3, 3*np.pi/4, 5*np.pi/6]
    gabor_filters_orient = [gabor_filter(size, lambda_val, theta, psi, sigma, gamma) 
                           for theta in orientations]
    
    # Generate filters with different phases
    phases = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4]
    gabor_filters_phase = [gabor_filter(size, lambda_val, theta, psi, sigma, gamma) 
                          for psi in phases]
    
    # Generate filters with different frequencies
    frequencies = [5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0, 22.5]
    gabor_filters_freq = [gabor_filter(size, lambda_val, theta, psi, sigma, gamma) 
                         for lambda_val in frequencies]
    
    # Visualize the filters
    fig, axes = plt.subplots(3, 8, figsize=(20, 8))
    
    # Plot filters with different orientations
    for i, (filt, orient) in enumerate(zip(gabor_filters_orient, orientations)):
        im = axes[0, i].imshow(filt, cmap='RdBu')
        axes[0, i].set_title(f'θ = {orient:.2f}')
        axes[0, i].axis('off')
    axes[0, 0].set_ylabel('Orientation')
    
    # Plot filters with different phases
    for i, (filt, phase) in enumerate(zip(gabor_filters_phase, phases)):
        im = axes[1, i].imshow(filt, cmap='RdBu')
        axes[1, i].set_title(f'ψ = {phase:.2f}')
        axes[1, i].axis('off')
    axes[1, 0].set_ylabel('Phase')
    
    # Plot filters with different frequencies
    for i, (filt, freq) in enumerate(zip(gabor_filters_freq, frequencies)):
        im = axes[2, i].imshow(filt, cmap='RdBu')
        axes[2, i].set_title(f'λ = {freq:.1f}')
        axes[2, i].axis('off')
    axes[2, 0].set_ylabel('Wavelength')
    
    plt.tight_layout()
    plt.suptitle('Gabor Filters with Different Parameters', fontsize=16, y=1.02)
    plt.show()
    
# Plot the Gabor filters
plot_gabor_filters()

### 4.1 Applying Gabor Filters to Images

Gabor filters are particularly effective at detecting oriented edges and textures in images. Let's apply our Gabor filters to the test image and see how they respond to different orientations:

In [ ]:
def apply_gabor_filter_bank(image, orientations, wavelengths, phases, sigma, gamma):
    """
    Apply a bank of Gabor filters to an image.
    
    Parameters:
    -----------
    image : ndarray
        Input image
    orientations : list
        List of orientations in radians
    wavelengths : list
        List of wavelengths
    phases : list
        List of phases in radians
    sigma : float
        Standard deviation of the Gaussian envelope
    gamma : float
        Spatial aspect ratio
    
    Returns:
    --------
    responses : dict
        Dictionary of filter responses
    """
    responses = {}
    size = min(64, min(image.shape))  # Size of the filter
    
    # Apply each filter to the image
    for theta in orientations:
        for lambda_val in wavelengths:
            for psi in phases:
                # Create the Gabor filter
                filt = gabor_filter(size, lambda_val, theta, psi, sigma, gamma)
                
                # Apply the filter
                response = signal.convolve2d(image, filt, mode='same', boundary='symm')
                
                # Store the response
                key = (theta, lambda_val, psi)
                responses[key] = response
    
    return responses

# Apply Gabor filters to the test image
orientations = [0, np.pi/4, np.pi/2, 3*np.pi/4]
wavelengths = [10.0]
phases = [0]
sigma = 5.0
gamma = 1.0

responses = apply_gabor_filter_bank(test_image, orientations, wavelengths, phases, sigma, gamma)

# Visualize the filter responses
fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Original image
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Filter responses
for i, theta in enumerate(orientations, 1):
    key = (theta, wavelengths[0], phases[0])
    response = responses[key]
    
    # Normalize for visualization
    response = (response - np.min(response)) / (np.max(response) - np.min(response))
    
    axes[i].imshow(response, cmap='gray')
    axes[i].set_title(f'Orientation: {theta:.2f} rad')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

### 4.2 Quadrature Pairs and Phase Invariance

A key concept in motion energy models is the use of quadrature pairs of Gabor filters. A quadrature pair consists of two filters with the same orientation, frequency, and envelope, but with phases that differ by 90 degrees (π/2 radians).

By combining the responses of a quadrature pair, we can achieve phase invariance, which is important for detecting motion regardless of the exact phase of the input signal.

In [ ]:
def create_quadrature_pair(size, lambda_val, theta, sigma, gamma):
    """
    Create a quadrature pair of Gabor filters.
    
    Parameters:
    -----------
    size : int
        Size of the filter (size x size)
    lambda_val : float
        Wavelength of the sinusoidal component
    theta : float
        Orientation of the Gabor filter in radians
    sigma : float
        Standard deviation of the Gaussian envelope
    gamma : float
        Spatial aspect ratio
    
    Returns:
    --------
    even_gabor : ndarray
        Even (cosine) Gabor filter
    odd_gabor : ndarray
        Odd (sine) Gabor filter
    """
    # Even (cosine) Gabor filter (phase = 0)
    even_gabor = gabor_filter(size, lambda_val, theta, 0, sigma, gamma)
    
    # Odd (sine) Gabor filter (phase = π/2)
    odd_gabor = gabor_filter(size, lambda_val, theta, np.pi/2, sigma, gamma)
    
    return even_gabor, odd_gabor

# Create and visualize a quadrature pair
size = 64
lambda_val = 10.0
theta = np.pi/4  # 45 degrees
sigma = 10.0
gamma = 1.0

even_gabor, odd_gabor = create_quadrature_pair(size, lambda_val, theta, sigma, gamma)

# Plot the quadrature pair
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Even Gabor
im1 = axes[0].imshow(even_gabor, cmap='RdBu')
axes[0].set_title('Even Gabor (Cosine Phase)')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0])

# Odd Gabor
im2 = axes[1].imshow(odd_gabor, cmap='RdBu')
axes[1].set_title('Odd Gabor (Sine Phase)')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1])

# Profile along the center
center = size // 2
axes[2].plot(even_gabor[center, :], 'b-', label='Even (Cosine)')
axes[2].plot(odd_gabor[center, :], 'r-', label='Odd (Sine)')
axes[2].set_title('Filter Profiles at Center')
axes[2].set_xlabel('Position')
axes[2].set_ylabel('Filter Response')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def apply_quadrature_pair(image, size, lambda_val, theta, sigma, gamma):
    """
    Apply a quadrature pair of Gabor filters to an image and compute the energy.
    
    Parameters:
    -----------
    image : ndarray
        Input image
    size : int
        Size of the filter
    lambda_val : float
        Wavelength of the sinusoidal component
    theta : float
        Orientation of the Gabor filter in radians
    sigma : float
        Standard deviation of the Gaussian envelope
    gamma : float
        Spatial aspect ratio
    
    Returns:
    --------
    even_response : ndarray
        Response to the even Gabor filter
    odd_response : ndarray
        Response to the odd Gabor filter
    energy : ndarray
        Energy (sum of squared responses)
    """
    # Create the quadrature pair
    even_gabor, odd_gabor = create_quadrature_pair(size, lambda_val, theta, sigma, gamma)
    
    # Apply the filters
    even_response = signal.convolve2d(image, even_gabor, mode='same', boundary='symm')
    odd_response = signal.convolve2d(image, odd_gabor, mode='same', boundary='symm')
    
    # Compute the energy (squared magnitude of the complex response)
    energy = even_response**2 + odd_response**2
    
    return even_response, odd_response, energy

# Apply quadrature pairs at different orientations
orientations = [0, np.pi/6, np.pi/3, np.pi/2, 2*np.pi/3, 5*np.pi/6]
size = 32
lambda_val = 8.0
sigma = 4.0
gamma = 1.0

# Collect responses
responses = {}
for theta in orientations:
    even_response, odd_response, energy = apply_quadrature_pair(
        test_image, size, lambda_val, theta, sigma, gamma)
    responses[theta] = (even_response, odd_response, energy)

# Visualize the responses
fig, axes = plt.subplots(len(orientations), 3, figsize=(12, 18))

for i, theta in enumerate(orientations):
    even_response, odd_response, energy = responses[theta]
    
    # Normalize for visualization
    even_response = (even_response - np.min(even_response)) / (np.max(even_response) - np.min(even_response))
    odd_response = (odd_response - np.min(odd_response)) / (np.max(odd_response) - np.min(odd_response))
    energy = energy / np.max(energy)
    
    # Even response
    axes[i, 0].imshow(even_response, cmap='gray')
    axes[i, 0].set_title(f'Even Response (θ = {theta:.2f})')
    axes[i, 0].axis('off')
    
    # Odd response
    axes[i, 1].imshow(odd_response, cmap='gray')
    axes[i, 1].set_title(f'Odd Response (θ = {theta:.2f})')
    axes[i, 1].axis('off')
    
    # Energy
    axes[i, 2].imshow(energy, cmap='viridis')
    axes[i, 2].set_title(f'Energy (θ = {theta:.2f})')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

## 5. Temporal Gabor Filters

For motion detection, we need to extend our Gabor filters to include the time dimension. A temporal Gabor filter can be defined similarly to the spatial Gabor filter, but with an additional dimension for time.

We can create a separable spatiotemporal Gabor filter by multiplying a spatial Gabor filter with a temporal Gabor filter.

In [ ]:
def temporal_gabor(length, omega, phi, sigma):
    """
    Create a 1D temporal Gabor filter.
    
    Parameters:
    -----------
    length : int
        Length of the filter
    omega : float
        Temporal frequency in cycles per sample
    phi : float
        Phase offset in radians
    sigma : float
        Standard deviation of the Gaussian envelope
    
    Returns:
    --------
    filter : ndarray
        1D temporal Gabor filter
    """
    t = np.arange(length) - length // 2
    gaussian = np.exp(-t**2 / (2 * sigma**2))
    sinusoid = np.cos(2 * np.pi * omega * t + phi)
    return gaussian * sinusoid

def spatiotemporal_gabor(spatial_size, temporal_length, lambda_val, theta, psi, sigma_spatial, gamma, omega, phi, sigma_temporal):
    """
    Create a separable spatiotemporal Gabor filter.
    
    Parameters:
    -----------
    spatial_size : int
        Size of the spatial filter (spatial_size x spatial_size)
    temporal_length : int
        Length of the temporal filter
    lambda_val : float
        Spatial wavelength
    theta : float
        Spatial orientation in radians
    psi : float
        Spatial phase offset in radians
    sigma_spatial : float
        Standard deviation of the spatial Gaussian envelope
    gamma : float
        Spatial aspect ratio
    omega : float
        Temporal frequency in cycles per sample
    phi : float
        Temporal phase offset in radians
    sigma_temporal : float
        Standard deviation of the temporal Gaussian envelope
    
    Returns:
    --------
    filter_3d : ndarray
        3D spatiotemporal Gabor filter (spatial_size x spatial_size x temporal_length)
    """
    # Create the spatial Gabor filter
    spatial_gabor = gabor_filter(spatial_size, lambda_val, theta, psi, sigma_spatial, gamma)
    
    # Create the temporal Gabor filter
    temp_gabor = temporal_gabor(temporal_length, omega, phi, sigma_temporal)
    
    # Create the 3D filter by multiplying the spatial and temporal filters
    filter_3d = np.zeros((spatial_size, spatial_size, temporal_length))
    for t in range(temporal_length):
        filter_3d[:, :, t] = spatial_gabor * temp_gabor[t]
    
    return filter_3d

# Create and visualize a spatiotemporal Gabor filter
spatial_size = 32
temporal_length = 16
lambda_val = 8.0
theta = np.pi/4
psi = 0.0
sigma_spatial = 4.0
gamma = 1.0
omega = 0.2
phi = 0.0
sigma_temporal = 4.0

filter_3d = spatiotemporal_gabor(
    spatial_size, temporal_length, lambda_val, theta, psi, sigma_spatial, gamma, omega, phi, sigma_temporal)

# Visualize the 3D filter by showing slices at different time points
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

time_points = np.linspace(0, temporal_length-1, 8, dtype=int)
for i, t in enumerate(time_points):
    im = axes[i].imshow(filter_3d[:, :, t], cmap='RdBu')
    axes[i].set_title(f'Time: {t}')
    axes[i].axis('off')
    plt.colorbar(im, ax=axes[i])

plt.tight_layout()
plt.suptitle('Spatiotemporal Gabor Filter at Different Time Points', fontsize=16, y=1.02)
plt.show()

## 6. Filters in the Context of Neural Processing

Filters, particularly Gabor filters, have a strong connection to the visual processing in the brain. The receptive fields of simple cells in the primary visual cortex (V1) can be well-approximated by Gabor filters.

### Receptive Fields in the Visual System

- **Retinal Ganglion Cells and LGN Cells**: Have circular center-surround receptive fields, which can be modeled as difference-of-Gaussians filters.
  
- **Simple Cells in V1**: Have oriented receptive fields, which can be modeled as Gabor filters. These cells are selective for orientation, spatial frequency, and phase.
  
- **Complex Cells in V1**: Are selective for orientation and spatial frequency, but not phase. This phase invariance can be modeled using quadrature pairs of Gabor filters.
  
- **Direction-Selective Cells**: Respond to motion in a specific direction. These can be modeled using spatiotemporal Gabor filters.

### Filter Banks and Population Coding

In the visual system, there are populations of neurons with receptive fields tuned to different orientations, spatial frequencies, and other parameters. This can be modeled using a filter bank, where each filter corresponds to a different neuron or group of neurons.

By combining the responses from multiple filters, the visual system can represent complex visual features and motion patterns. This is the basis for motion energy models, which we'll explore in more detail in later sections.

## 7. Summary

In this section, we've explored filtering, a fundamental operation in signal processing and a core component of motion energy models. Here's a summary of what we've learned:

1. **Filtering basics**: Filters can selectively enhance or suppress certain aspects of a signal. They are characterized by their impulse response or frequency response.

2. **Types of filters**: We explored low-pass, high-pass, band-pass, and band-stop filters, and saw how they affect signals differently.

3. **Spatial filtering**: We applied various spatial filters to images, including blur filters, edge detection filters, and more.

4. **Temporal filtering**: We explored how filters can be applied to time-varying signals to enhance or suppress specific temporal features.

5. **Gabor filters**: We implemented and visualized Gabor filters, which play a crucial role in motion energy models due to their similarity to receptive fields in the visual cortex.

6. **Quadrature pairs**: We learned about quadrature pairs of Gabor filters and how they can achieve phase invariance, which is important for motion detection.

7. **Spatiotemporal filters**: We extended our Gabor filters to include the time dimension, creating filters that can detect motion in specific directions.

8. **Neural connection**: We discussed the relationship between filters and receptive fields in the visual system, highlighting the biological relevance of our computational models.

In the next section, we'll explore spatiotemporal representations of motion and see how these filters can be used to detect and analyze motion patterns.

## Further Reading

- Adelson, E. H., & Bergen, J. R. (1985). Spatiotemporal energy models for the perception of motion. Journal of the Optical Society of America A, 2(2), 284-299.

- Field, D. J. (1987). Relations between the statistics of natural images and the response properties of cortical cells. Journal of the Optical Society of America A, 4(12), 2379-2394.

- Jones, J. P., & Palmer, L. A. (1987). An evaluation of the two-dimensional Gabor filter model of simple receptive fields in cat striate cortex. Journal of Neurophysiology, 58(6), 1233-1258.

- Marčelja, S. (1980). Mathematical description of the responses of simple cortical cells. Journal of the Optical Society of America, 70(11), 1297-1300.

- Daugman, J. G. (1985). Uncertainty relation for resolution in space, spatial frequency, and orientation optimized by two-dimensional visual cortical filters. Journal of the Optical Society of America A, 2(7), 1160-1169.